In [ ]:
# Импорт необходимых библиотек
from __future__ import print_function
import numpy as np
import mxnet as mx
from mxnet import nd, autograd, gluon

# Случайный сид
mx.random.seed(666)

# Спецификация метода вычислений
ctx = mx.cpu()

# Гиперпараметры
batch_size = 64
num_inputs = 784
num_outputs = 10

# 
def transform(data, label):
    return nd.transpose(data.astype(np.float32), (2,0,1))/255, label.astype(np.float32)

# Загрузка данных
train_data = gluon.data.DataLoader(gluon.data.vision(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = gluon.data.DataLoader(gluon.data.vision(train=False, transform=transform),
                                     batch_size, shuffle=False)


# Инициализация нейронной сети
net = gluon.nn.Sequential()

# Построение сети
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=32, kernel_size=3, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Conv2D(channels=64, kernel_size=3, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Conv2D(channels=128, kernel_size=3, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    # https://github.com/apache/incubator-mxnet/issues/8176#issuecomment-357225803
    net.add(gluon.nn.Conv2DTranspose(channels=64, kernel_size=3, activation='relu'))
    net.add(gluon.nn.Conv2DTranspose(channels=32, kernel_size=3, activation='relu'))
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(128, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))
    
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

# Оптимизатор
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

# Оценка точности
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

epochs = 10
smoothing_constant = .01

for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + smoothing_constant * curr_loss)

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))